# Imports

In [1]:
import pandas as pd
import cv2
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import torch
from torch import topk
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import os
from collections import defaultdict
from sklearn.model_selection import KFold
import numpy as np
from tqdm import tqdm
import math
import time

In [2]:
%pip install wandb -q
import wandb
WANDB_NOTEBOOK_NAME="first_model_1"
wandb.login()

Note: you may need to restart the kernel to use updated packages.


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: maryem_hajji. Use `wandb login --relogin` to force relogin


True

# Training csv file

In [3]:
train_images_dir="train_images"
test_images_dir=""
train_csv_dir="~/train.csv"
test_csv_dir=""

In [4]:
def indiv_image_dir(photo_name):
    return f'{train_images_dir}/{photo_name}'

In [5]:
training_file=pd.read_csv(f"{train_csv_dir}")
print(training_file.head())

training_file["image_path"]= training_file["image"].apply(indiv_image_dir)
print(training_file["image_path"].head())

                image             species individual_id
0  00021adfb725ed.jpg  melon_headed_whale  cadddb1636b9
1  000562241d384d.jpg      humpback_whale  1a71fbb72250
2  0007c33415ce37.jpg  false_killer_whale  60008f293a2b
3  0007d9bca26a99.jpg  bottlenose_dolphin  4b00fe572063
4  00087baf5cef7a.jpg      humpback_whale  8e5253662392
0    train_images/00021adfb725ed.jpg
1    train_images/000562241d384d.jpg
2    train_images/0007c33415ce37.jpg
3    train_images/0007d9bca26a99.jpg
4    train_images/00087baf5cef7a.jpg
Name: image_path, dtype: object


# Setting the Encoder: list of labels' numbers 

In [6]:
encoder=preprocessing.LabelEncoder()
list_of_all_ids=training_file['individual_id']

ids_classes=encoder.fit_transform(list_of_all_ids).astype('int32')
number_classes=max(ids_classes)+1 #les indices de classes commencent à 0
training_file["ids_classes"]=ids_classes

# Dataset

In [7]:
class WhaleDataset(data.Dataset):
    def __init__(self, csv):
        
        self.csv = csv
        self.image_list = csv["image_path"]
        self.classes_map=csv["ids_classes"]
        self.resize = transforms.Compose([
                                         transforms.ToPILImage(),
                                         transforms.Resize((224,224)),
                                         transforms.ToTensor()
        ])
     
    def __len__(self):
        return len(self.csv)
    
    def __getitem__(self, idx):
        
        img = torchvision.io.read_image(self.image_list.iloc[idx]).float()
      
        if img.shape[0] == 1:
            img = torch.cat((img,img,img))
        #img=cv2.imread(img)
        img = self.resize(img)
        #img=torch.tensor(img)
        label = self.classes_map.iloc[idx]
        label=torch.tensor(label,dtype=torch.long)
        return {"image" :img, "label": label}

# Training: cross validation

In [8]:
class ConvNet_Classifier(nn.Module):
    def __init__(self, conv_net, output_size, num_classes):
        super().__init__()
        self.conv_net = conv_net
        self.linear = nn.Linear(output_size, num_classes)
    def forward(self, batch_data):
        x = self.conv_net(batch_data)
        x = self.linear(x)
        return x

In [9]:
def loss_function(outputs, labels):
    return nn.CrossEntropyLoss()(outputs, labels)


In [12]:
# Dividing the dataset into 3 folds
num_folds=3
fold=StratifiedKFold(n_splits=num_folds,shuffle=True,random_state=1)
L=enumerate(fold.split(training_file.image,training_file.individual_id))

for i, (train_index,validation_index) in L :
    training_file.loc[validation_index,"is_validation"]=i+1


/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


In [10]:
def train(model,device,dataloader,optimizer,epoch,scheduler,batch_size):
    model.train()
    print("starting training")
    
    data_len=0
    inter_loss=0.0
    
    temp=tqdm(enumerate(dataloader))
    
    for batch, _data in temp:
        images=_data['image'].to(device, dtype=torch.float)
        labels=_data['label'].to(device, dtype=torch.long)
        
        optimizer.zero_grad()
        
        outputs=model(images)

        loss=loss_function(outputs,labels)
        loss.backward()
        
        optimizer.step()
        
        
        if scheduler is not None:
            scheduler.step()
        
        inter_loss += (loss.item()*batch_size)
        data_len += batch_size
        
        final_loss = inter_loss / data_len
        wandb.log({"train loss": final_loss})
    
    
    return final_loss

In [11]:
def validation(model,dataloader,batch_size,device,epoch):
    
    model.eval()
    data_len=0.0
    inter_loss=0.0

    temp=tqdm(enumerate(dataloader))
    
    for batch, _data in temp:
        images=_data['image'].to(device, dtype=torch.float)
        labels=_data['label'].to(device, dtype=torch.long)
        
        
        data_len += batch_size
        outputs=model(images)
        
        loss=loss_function(outputs,labels)
        inter_loss += (loss.item()*batch_size)
        final_loss = inter_loss / data_len
        
        wandb.log({"validation loss": final_loss})
        

    return final_loss


In [13]:
batch_size=256
data_set=WhaleDataset(training_file)

In [14]:
def training(model,optimizer,scheduler,device, batch_size=batch_size, epochs=1,data_set=training_file):
    wandb.watch(model,log_freq=10)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

    print(device)
    model.to(device)

    for i in range(num_folds):
    
        kwargs = {'num_workers': 6, 'pin_memory': True} if use_cuda else {}
        
        train_set=WhaleDataset(data_set[data_set["is_validation"]!=i+1])
        validation_set=WhaleDataset(data_set[data_set["is_validation"]==i+1])
        
        train_loader = data.DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True,**kwargs)
        validation_loader = data.DataLoader(dataset=validation_set,batch_size=batch_size,shuffle=True,**kwargs)

        print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

        optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-6)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 1000)

        start = time.time()

        for epoch in range(1, epochs + 1):
            train_loss=train(model,device,train_loader,optimizer,epoch,scheduler,batch_size)
            validation_loss=validation(model,validation_loader,batch_size,device,epoch)
            
            torch.save(model.state_dict(), "resnet.pt")

        end = time.time()
        time_elapsed = end - start
        print(time_elapsed)
    

In [15]:
track = wandb.init(project='Cross_validation_HappyWhale', 
                 job_type='Train and Validation',
                 tags=['resnet18()'],
                 anonymous='must')


In [ ]:
conv_model = torchvision.models.resnet18(pretrained = True)
model = ConvNet_Classifier(conv_model,1000,number_classes)

In [17]:
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-6)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 1000)

In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
training(model,optimizer,scheduler,device=device,epochs=3)

cuda:0
Num Model Parameters 27292099
starting training


150it [13:40,  5.47s/it, Epoch=1, LR=0.0001, Train_Loss=8.63]
75it [07:13,  5.78s/it, Epoch=1, LR=0.0001, validation_Loss=7.67]


starting training


150it [13:37,  5.45s/it, Epoch=2, LR=0.0001, Train_Loss=6.68]
75it [07:04,  5.65s/it, Epoch=2, LR=0.0001, validation_Loss=7.17]


starting training


150it [13:37,  5.45s/it, Epoch=3, LR=0.0001, Train_Loss=5.16]
75it [07:00,  5.61s/it, Epoch=3, LR=0.0001, validation_Loss=7.01]


Training complete in 1h 2m 17s
Num Model Parameters 27292099
starting training


150it [13:49,  5.53s/it, Epoch=1, LR=0.0001, Train_Loss=5.94]
75it [06:56,  5.56s/it, Epoch=1, LR=0.0001, validation_Loss=4.92]


starting training


150it [13:25,  5.37s/it, Epoch=2, LR=0.0001, Train_Loss=4.45]
47it [04:16,  3.62s/it, Epoch=2, LR=0.0001, validation_Loss=4.99]

In [ ]:
track.finish()

In [ ]:
torch.save(model.state_dict(), "model_HappyWhale.pt")